<a href="https://colab.research.google.com/github/NahinAlam001/CSE-465/blob/model/Welcome_to_Colaboratory.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import os
import cv2
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from torchvision import transforms
from torch.utils.data import DataLoader, Dataset
import torch

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
class SkinLesionDataset(Dataset):
    def __init__(self, image_paths, labels=None, transform=None):
        self.image_paths = image_paths
        self.labels = labels
        self.transform = transform

    def __len__(self):
        return len(self.image_paths)

    def __getitem__(self, idx):
        image = cv2.imread(self.image_paths[idx])
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        if self.transform:
            image = self.transform(image)
        if self.labels is not None:
            label = self.labels[idx]
            return image, label
        return image

In [4]:
def load_ham10000_dataset(img_dir, csv_file):
    df = pd.read_csv(csv_file)
    image_paths = [os.path.join(img_dir, f"{image_id}.jpg") for image_id in df['image_id']]
    labels = pd.get_dummies(df['dx']).values
    return image_paths, labels

In [7]:
ham10000_img_dir = '/content/drive/MyDrive/PH2data/data'
ham10000_csv_file = '/content/drive/MyDrive/PH2data/data.csv'

In [8]:
ham10000_image_paths, ham10000_labels = load_ham10000_dataset(ham10000_img_dir, ham10000_csv_file)

In [9]:
X_train_paths, X_test_paths, y_train, y_test = train_test_split(ham10000_image_paths, ham10000_labels, test_size=0.2, random_state=42)

In [10]:
transform = transforms.Compose([
    transforms.ToPILImage(),
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

In [11]:
train_dataset = SkinLesionDataset(X_train_paths, y_train, transform=transform)
test_dataset = SkinLesionDataset(X_test_paths, y_test, transform=transform)

In [19]:
train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=64, shuffle=False)

In [20]:
batch_img, batch_label = next(iter(train_loader))
batch_img.shape, batch_label.shape

(torch.Size([64, 3, 224, 224]), torch.Size([64, 3]))

In [21]:
import torch.nn as nn
import torch.optim as optim
from torchvision import models

# Define CNN model
class CNNModel(nn.Module):
    def __init__(self, num_classes):
        super(CNNModel, self).__init__()
        self.features = nn.Sequential(
            nn.Conv2d(3, 16, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=2, stride=2),
            nn.Conv2d(16, 32, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=2, stride=2),
            nn.Conv2d(32, 64, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=2, stride=2),
            nn.Conv2d(64, 128, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=2, stride=2)
        )
        self.classifier = nn.Sequential(
            nn.Flatten(),
            nn.Linear(128 * 14 * 14, 64),
            nn.ReLU(inplace=True),
            nn.Dropout(0.5),
            nn.Linear(64, 32),
            nn.ReLU(inplace=True),
            nn.Linear(32, num_classes)
        )

    def forward(self, x):
        x = self.features(x)
        x = self.classifier(x)
        return x

# Instantiate the model, define loss function and optimizer
num_classes = 3
model = CNNModel(num_classes=num_classes)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Move model to GPU if available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

CNNModel(
  (features): Sequential(
    (0): Conv2d(3, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU(inplace=True)
    (2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (3): Conv2d(16, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (4): ReLU(inplace=True)
    (5): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (6): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (7): ReLU(inplace=True)
    (8): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (9): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (10): ReLU(inplace=True)
    (11): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (classifier): Sequential(
    (0): Flatten(start_dim=1, end_dim=-1)
    (1): Linear(in_features=25088, out_features=64, bias=True)
    (2): ReLU(inplace=True)
    (3): Dropout(p=0.5, inplace=False)
    (4): L

In [22]:
# Training function
def train_model(model, dataloader, criterion, optimizer, device, num_epochs=20):
    model.train()
    for epoch in range(num_epochs):
        running_loss = 0.0
        for images, labels in dataloader:
            images = images.to(device)
            labels = labels.to(device)

            optimizer.zero_grad()
            outputs = model(images)
            loss = criterion(outputs, torch.max(labels, 1)[1])
            loss.backward()
            optimizer.step()

            running_loss += loss.item() * images.size(0)

        epoch_loss = running_loss / len(dataloader.dataset)
        print(f"Epoch [{epoch+1}/{num_epochs}], Loss: {epoch_loss:.4f}")

# Train the model
train_model(model, train_loader, criterion, optimizer, device)

Epoch [1/20], Loss: 1.1199
Epoch [2/20], Loss: 1.0619
Epoch [3/20], Loss: 1.0126
Epoch [4/20], Loss: 0.9241
Epoch [5/20], Loss: 0.8687
Epoch [6/20], Loss: 0.8803
Epoch [7/20], Loss: 0.8608
Epoch [8/20], Loss: 0.8009
Epoch [9/20], Loss: 0.7711
Epoch [10/20], Loss: 0.7697
Epoch [11/20], Loss: 0.7337
Epoch [12/20], Loss: 0.7294
Epoch [13/20], Loss: 0.7183
Epoch [14/20], Loss: 0.6445
Epoch [15/20], Loss: 0.6537
Epoch [16/20], Loss: 0.6899
Epoch [17/20], Loss: 0.6661
Epoch [18/20], Loss: 0.6690
Epoch [19/20], Loss: 0.6186
Epoch [20/20], Loss: 0.6217


##Implementing the Segmentation Model

In [23]:
# Assuming you have the Segment Anything Model implemented or downloaded
# For demonstration, we use a simple U-Net model for segmentation

class UNet(nn.Module):
    def __init__(self):
        super(UNet, self).__init__()
        # Encoder
        self.enc1 = self.conv_block(3, 64)
        self.enc2 = self.conv_block(64, 128)
        self.enc3 = self.conv_block(128, 256)
        self.enc4 = self.conv_block(256, 512)

        # Decoder
        self.dec4 = self.upconv_block(512, 256)
        self.dec3 = self.upconv_block(256, 128)
        self.dec2 = self.upconv_block(128, 64)
        self.dec1 = nn.Sequential(
            nn.ConvTranspose2d(64, 32, kernel_size=2, stride=2),
            nn.Conv2d(32, 1, kernel_size=1)
        )

    def conv_block(self, in_channels, out_channels):
        return nn.Sequential(
            nn.Conv2d(in_channels, out_channels, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.Conv2d(out_channels, out_channels, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=2, stride=2)
        )

    def upconv_block(self, in_channels, out_channels):
        return nn.Sequential(
            nn.ConvTranspose2d(in_channels, out_channels, kernel_size=2, stride=2),
            nn.Conv2d(out_channels, out_channels, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.Conv2d(out_channels, out_channels, kernel_size=3, padding=1),
            nn.ReLU(inplace=True)
        )

    def forward(self, x):
        e1 = self.enc1(x)
        e2 = self.enc2(e1)
        e3 = self.enc3(e2)
        e4 = self.enc4(e3)

        d4 = self.dec4(e4)
        d3 = self.dec3(d4 + e3)
        d2 = self.dec2(d3 + e2)
        d1 = self.dec1(d2 + e1)

        return d1

# Instantiate the U-Net model, define loss function and optimizer
unet = UNet()
unet = unet.to(device)
seg_criterion = nn.BCEWithLogitsLoss()
seg_optimizer = optim.Adam(unet.parameters(), lr=0.003)